<RNN을 이용한 텍스트 데이터 분류>

In [4]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

# history를 그래프로 그리기 위해 사용
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metrics],'')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()
    


1. 입력할 Datasets 준비
- 사용할 Datasets : IMDB(Internet Movie Database) 영화리뷰를 모다 놓은 데이터셋, 각 리뷰가 긍정인지 부정인지에 대한 label 포함
- 어떤 리뷰가 긍정/부정인지를 분류하는 binary classification 문제를 학습하기 위해 많이 사용
- text는 대표적인 sequential data로 RNN을 사용하기 적잡
- tfds를 통해 간단히 load 및 전처리하여 사용 가능

In [15]:
# 입력 데이터 준비
dataset, info = tfds.load('imdb_reviews/subwords8k',with_info=True, as_supervised=True)
print(dataset,info)

train_dataset, test_dataset = dataset['train'],dataset['test']

{'train': <PrefetchDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>, 'test': <PrefetchDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>, 'unsupervised': <PrefetchDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>} tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/subwords8k/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Uses `tfds.deprecated.text.SubwordTextEncoder` with 8k vocab size
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='C:\\Users\\Admin\\tensorflow_datasets\\imdb_reviews\\subwords8k\\1.0.0',
    download_size=80.23 MiB,
    dataset_size=54.72 MiB,
    features=Fea

In [17]:
# info에는 인코딩정보가 담기며, 컴퓨터가 인식하는 단어수를 출력한다.
encoder = info.features['text'].encoder
print("Vocabulary size : {}".format(encoder.vocab_size))


Vocabulary size : 8185


In [20]:
sample_string = "Hello Tensorflow"

encoded_string = encoder.encode(sample_string)
print("Encoded string is : {}".format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string is ; {}'.format(original_string))

# 원래 문장과 encode-decode 과정을 통과한 문장은 항상 동일함.
assert original_string == sample_string


Encoded string is : [4025, 222, 6307, 2327, 2934]
The original string is ; Hello Tensorflow


In [21]:
for index in encoded_string:
    print('{} ---> {}'.format(index, encoder.decode([index])))

4025 ---> Hell
222 ---> o 
6307 ---> Ten
2327 ---> sor
2934 ---> flow


[Tokenizer] : text를 token(정수)단위로 표현해주는 메소드
1. 영어에서 알파벳 하나씩 token단위로 변환하는 방법(character level)
2. 음절 단위로 변환
3. 단어 단위로 변환(word level)

한국어 문장을 이용하기 위해서는 한국어 전용 Tokenizer가 필요함. 

In [24]:
# 학습을 위한 데이터셋 준비
num_train = len(list(train_dataset))
num_test = len(list(test_dataset))

print('학습 데이터 수 : ',num_train)
print('테스트 데이터 수 : ',num_test)

BUFFER_SIZE = 10000
BATCH_SIZE = 64

학습 데이터 수 :  25000
테스트 데이터 수 :  25000


In [ ]:
# 테스트 데이터셋은 반으로 나누어서 1/2는 validation dataset으로 사용
valid_dataset = test_dataset.take(num_test//2)
test_dataset = test_dataset.skip(num_test//2)

# 학습 시간이 오래걸리므로 학습 데이터를 1/5만 사용
train_dataset = train_dataset.take(num_train//5)


